MixUp

In [ ]:
#hide
!fusermount -u drive
!google-drive-ocamlfuse drive
from google.colab import drive
drive.mount('/content/drive')

fusermount: failed to unmount /content/drive: No such file or directory
/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
#!pip install torch
#!pip install pytorch
#!pip install torchvision
#!pip install fastai
!pip3 install git+https://github.com/fastai/fastai.git 
!pip install -Uqq fastbook

In [ ]:
#hide
import PIL
from PIL import Image 
import os
import requests
import torch
import pandas as pd
import fastbook
from fastai.vision.widgets import *
fastbook.setup_book()

In [ ]:
#hide
import requests
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
#path = Path('gdrive/MyDrive/PBA Softwareudvikling/AI Machinelearning/AI_reCaptcha v2/recaptcha-dataset-main/trimmed')
rootPath = glob.glob(os.path.join(os.getcwd(), "gdrive/MyDrive/**/AI_reCaptcha v2/"), recursive=true)[0]
path = os.path.join(rootPath, "recaptcha-dataset-main/trimmed/")
path

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
def get_y(r): return L(parent_label(r))
#Vi prøvede først at lave vores egen måde at loade filer ind for at begrænse datasettet
# def get_image_files_by_size(path, sample_size = 200): 
#   return list(get_image_files(path))[:sample_size]

In [ ]:
ys = []
for i in get_image_files(path):
  ys.append(get_y(i))
ys

In [ ]:
dblock = DataBlock(
    blocks = (ImageBlock, MultiCategoryBlock), 
    get_items = get_image_files, 
    get_y = get_y, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=RandomResizedCrop(128, min_scale=0.35))
dls = dblock.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=200, nrows=10)

In [ ]:
learn = cnn_learner(dls, 
                    resnet50, 
                    metrics=accuracy_multi, 
                    cbs=MixUp)

In [ ]:
learn.fine_tune(20, base_lr=8.32e-03)

In [ ]:
#learn.fit_one_cycle(8.32e-03)

In [ ]:
lr_min,lr_steep = learn.lr_find()
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
#hide_output
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])
#hide_output
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn.predict(img )

#hide_output
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]}'
lbl_pred

In [ ]:
learn.dls.vocab

In [ ]:
probs

In [ ]:
pred_idx